In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
! ls

drive  sample_data


In [3]:
import pandas as pd
import numpy as np

In [4]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [5]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [6]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [7]:
# function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# function to load the file
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'drive/My Drive/HAR/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).values
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [8]:
# creates one hot encoding of output
def load_y(subset):

    filename = f'drive/My Drive/HAR/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).values

In [9]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [10]:
# Importing libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout

In [11]:
# Initializing parameters
epochs = 26
batch_size = 16
n_hidden = 32

In [12]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [13]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [14]:
X_train.shape

(7352, 128, 9)

In [15]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [ ]:
print(pd.DataFrame(X_train[0]))

            0         1         2  ...         6         7         8
0    0.000181  0.010767  0.055561  ...  1.012817 -0.123217  0.102934
1    0.010139  0.006579  0.055125  ...  1.022833 -0.126876  0.105687
2    0.009276  0.008929  0.048405  ...  1.022028 -0.124004  0.102102
3    0.005066  0.007489  0.049775  ...  1.017877 -0.124928  0.106553
4    0.010810  0.006141  0.043013  ...  1.023680 -0.125767  0.102814
..        ...       ...       ...  ...       ...       ...       ...
123  0.000228 -0.002929 -0.003412  ...  1.019815 -0.127010  0.094843
124 -0.000300 -0.002023  0.000359  ...  1.019290 -0.126185  0.098350
125 -0.001147  0.000171  0.002648  ...  1.018445 -0.124070  0.100385
126 -0.000222  0.001574  0.002381  ...  1.019372 -0.122745  0.099874
127  0.001576  0.003070 -0.002270  ...  1.021171 -0.121326  0.094987

[128 rows x 9 columns]


- Defining the Architecture of LSTM

In [16]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                5376      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
            optimizer='rmsprop',
              metrics=['accuracy'])

In [19]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs,verbose = 1)

Epoch 1/26
460/460 [==============================] - 4s 8ms/step - loss: 1.2101 - accuracy: 0.5027 - val_loss: 0.8858 - val_accuracy: 0.6200
Epoch 2/26
460/460 [==============================] - 3s 7ms/step - loss: 0.8483 - accuracy: 0.6408 - val_loss: 0.8356 - val_accuracy: 0.6692
Epoch 3/26
460/460 [==============================] - 3s 7ms/step - loss: 0.6836 - accuracy: 0.7178 - val_loss: 0.6666 - val_accuracy: 0.7258
Epoch 4/26
460/460 [==============================] - 3s 7ms/step - loss: 0.4910 - accuracy: 0.8271 - val_loss: 0.5949 - val_accuracy: 0.8107
Epoch 5/26
460/460 [==============================] - 3s 7ms/step - loss: 0.3819 - accuracy: 0.8818 - val_loss: 0.4966 - val_accuracy: 0.8517
Epoch 6/26
460/460 [==============================] - 3s 7ms/step - loss: 0.2997 - accuracy: 0.9090 - val_loss: 0.4422 - val_accuracy: 0.8415
Epoch 7/26
460/460 [==============================] - 3s 7ms/step - loss: 0.2722 - accuracy: 0.9158 - val_loss: 0.3817 - val_accuracy: 0.8829
Epoch 

In [20]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 508        0  ...                   0                15
SITTING                  0      415  ...                   0                 3
STANDING                 0      112  ...                   0                 0
WALKING                  0        0  ...                   2                 0
WALKING_DOWNSTAIRS       0        0  ...                 396                 8
WALKING_UPSTAIRS         0        0  ...                   6               435

[6 rows x 6 columns]


In [21]:
score = model.evaluate(X_test, Y_test)

93/93 [==============================] - 0s 4ms/step - loss: 0.4401 - accuracy: 0.9050


In [22]:
score

[0.44011443853378296, 0.9049881100654602]

In [23]:
b = np.array([X_test[1]])

In [24]:
import tensorflow as tf
tf.__version__
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpo0p_d3ad/assets


INFO:tensorflow:Assets written to: /tmp/tmpo0p_d3ad/assets


In [25]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()
interpreter

In [26]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']
b= tf.convert_to_tensor(b, np.float32)
interpreter.set_tensor(input_details[0]['index'], b)
interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[1.6027659e-02 3.2281280e-03 1.0650456e-03 8.6243457e-01 9.9481618e-01
  1.9875169e-04]]


In [27]:
print(np.argmax(output_data))

4


In [28]:
input_details = interpreter.get_input_details()

In [29]:
print(input_details)

[{'name': 'serving_default_lstm_input:0', 'index': 0, 'shape': array([  1, 128,   9], dtype=int32), 'shape_signature': array([ -1, 128,   9], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
